Intersection Over Union(IOU) Implementation

In [ ]:
import numpy as np
from PIL import Image, ImageDraw, ImageFont
import supervision as sv
from shapely.geometry import box

def yolo_to_absolute_bbox(yolo_bbox, img_width, img_height):
    x_center, y_center, width, height = yolo_bbox
    x_center *= img_width
    y_center *= img_height
    width *= img_width
    height *= img_height
    
    x_min = x_center - width / 2
    y_min = y_center - height / 2
    x_max = x_center + width / 2
    y_max = y_center + height / 2
    
    return x_min, y_min, x_max, y_max

def compute_iou_shapely(yolo_bbox1, yolo_bbox2, img_width, img_height):

    x_min1, y_min1, x_max1, y_max1 = yolo_to_absolute_bbox(yolo_bbox1, img_width, img_height)
    x_min2, y_min2, x_max2, y_max2 = yolo_to_absolute_bbox(yolo_bbox2, img_width, img_height)
    
    box1 = box(x_min1, y_min1, x_max1, y_max1)
    box2 = box(x_min2, y_min2, x_max2, y_max2)
    
    intersection = box1.intersection(box2).area
    union = box1.union(box2).area
    
    iou = intersection / union if union > 0 else 0
    return iou

def compute_iou_supervision(yolo_bbox1, yolo_bbox2, img_width, img_height):

    abs_bbox1 = yolo_to_absolute_bbox(yolo_bbox1, img_width, img_height)
    abs_bbox2 = yolo_to_absolute_bbox(yolo_bbox2, img_width, img_height)

    bboxes1 = np.array([[abs_bbox1[1], abs_bbox1[0], abs_bbox1[3], abs_bbox1[2]]])  
    bboxes2 = np.array([[abs_bbox2[1], abs_bbox2[0], abs_bbox2[3], abs_bbox2[2]]])  
    iou = sv.detection.utils.box_iou_batch(bboxes1, bboxes2)[0, 0]
    return iou

def draw_bboxes_on_image(image_path, label_path, output_image_path):

    img = Image.open(image_path)
    img_width, img_height = img.size

    img_copy = img.copy()
    draw = ImageDraw.Draw(img_copy)

    with open(label_path, 'r') as f:
        labels = f.readlines()

    yolo_bbox7 = tuple(map(float, labels[6].strip().split()[1:]))  
    yolo_bbox8 = tuple(map(float, labels[7].strip().split()[1:]))  

    iou_shapely = compute_iou_shapely(yolo_bbox7, yolo_bbox8, img_width, img_height)
    iou_supervision = compute_iou_supervision(yolo_bbox7, yolo_bbox8, img_width, img_height)

    print(f"IoU (Shapely): {iou_shapely:.4f}")
    print(f"IoU (Supervision): {iou_supervision:.4f}")

    for i, label in enumerate(labels):
        label_info = label.strip().split()  
        class_id = int(label_info[0])  
        yolo_bbox = tuple(map(float, label_info[1:]))  

        x_min, y_min, x_max, y_max = yolo_to_absolute_bbox(yolo_bbox, img_width, img_height)

        if i == 6: 
            draw.rectangle([x_min, y_min, x_max, y_max], outline="green", width=3)
        elif i == 7:  
            draw.rectangle([x_min, y_min, x_max, y_max], outline="blue", width=3)
        else:
            draw.rectangle([x_min, y_min, x_max, y_max], outline="red", width=3)

        draw.text((x_min, y_min - 10), f"ID: {i+1}", fill="red")

    img_copy.save(output_image_path)
    img_copy.show()  

image_path = "data/images/native/solarpanels_native_3__x0_4414_y0_3879_dxdy_416.tif"
label_path = "data/labels/native/solarpanels_native_3__x0_4414_y0_3879_dxdy_416.txt"  
output_image_path = "output_image_for_iou.jpg"

draw_bboxes_on_image(image_path, label_path, output_image_path)


IoU (Shapely): 0.0515
IoU (Supervision): 0.0515


Average Precision(AP) Implementation

In [1]:
import numpy as np
import random
from shapely.geometry import box
import matplotlib.pyplot as plt
from sklearn.metrics import precision_recall_curve, auc

def generate_random_boxes(num_boxes=10, img_size=100, box_size=20):
    boxes = []
    for _ in range(num_boxes):
        x_min = random.randint(0, img_size - box_size)
        y_min = random.randint(0, img_size - box_size)
        x_max, y_max = x_min + box_size, y_min + box_size
        boxes.append((x_min, y_min, x_max, y_max))
    return boxes

def compute_iou(boxA, boxB):
    polyA = box(*boxA)
    polyB = box(*boxB)
    intersection = polyA.intersection(polyB).area
    union = polyA.union(polyB).area
    return intersection / union if union > 0 else 0

def match_boxes(gt_boxes, pred_boxes, iou_threshold=0.5):
    matches = []
    for pred in pred_boxes:
        best_iou = 0
        for gt in gt_boxes:
            iou = compute_iou(gt, pred)
            if iou > best_iou:
                best_iou = iou
        matches.append((best_iou >= iou_threshold, best_iou))
    return matches

def compute_ap_voc_11point(precisions, recalls):
    recall_levels = np.linspace(0, 1, 11)
    ap = 0
    for recall_level in recall_levels:
        max_prec = max(precisions[recalls >= recall_level], default=0)
        ap += max_prec
    return ap / 11

def compute_ap_coco_101point(precisions, recalls):
    recall_levels = np.linspace(0, 1, 101)
    ap = np.mean([max(precisions[recalls >= r], default=0) for r in recall_levels])
    return ap

def compute_ap_auc(precisions, recalls):
    return auc(recalls, precisions)

def evaluate_ap_methods():
    num_images = 10
    ap_results = {"VOC_11": [], "COCO_101": [], "AUC_PR": []}
    
    for _ in range(num_images):
        gt_boxes = generate_random_boxes()
        pred_boxes = generate_random_boxes()
        conf_scores = np.random.rand(len(pred_boxes))  
        
        matches = match_boxes(gt_boxes, pred_boxes)
        matches.sort(key=lambda x: x[1], reverse=True)  
        
        tps = np.array([1 if match[0] else 0 for match in matches])
        fps = 1 - tps
        tps = np.cumsum(tps)
        fps = np.cumsum(fps)
        recalls = tps / len(gt_boxes)
        precisions = tps / (tps + fps)
        
        ap_voc = compute_ap_voc_11point(precisions, recalls)
        ap_coco = compute_ap_coco_101point(precisions, recalls)
        ap_auc = compute_ap_auc(precisions, recalls)
        
        ap_results["VOC_11"].append(ap_voc)
        ap_results["COCO_101"].append(ap_coco)
        ap_results["AUC_PR"].append(ap_auc)
    
    return {method: np.mean(values) for method, values in ap_results.items()}

ap_values = evaluate_ap_methods()
print("Average Precision (AP50) comparison:")
for method, value in ap_values.items():
    print(f"{method}: {value:.4f}")


Average Precision (AP50) comparison:
VOC_11: 0.1455
COCO_101: 0.1248
AUC_PR: 0.0600
